# Configuration

In [ ]:
# Pre-requisite Configuration
# ------------------------------------------------------------------------------ #
# For testing
# ------------------------------------------------------------------------------ #

%load_ext autoreload
%autoreload 2

import os 
from modules.util.config import get_config_by_id, get_config_global
from modules.util.database import SQLAlchemyClient
from modules.util.helpers import Logger


# ------------------------------------------------------------------------------ #
# Configuration
# ------------------------------------------------------------------------------ #

CONFIG_ID = 'dca-test'
TRANSFORM = get_config_by_id(CONFIG_ID)["load"]["indicator"]
EXTRACTION_DIR = TRANSFORM["directory"]
REPORTS_DIR = f"{EXTRACTION_DIR}/reports"
CONFIG_GLOBAL = get_config_global().get('indicators').get('transform')

# ------------------------------------------------------------------------------ #
# Create directories
# ------------------------------------------------------------------------------ #
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)

# ------------------------------------------------------------------------------ #
# Logger
# ------------------------------------------------------------------------------ #

log = Logger.get_logger(CONFIG_ID)

Logger.blank_line(log)
log.info("** LOAD - INDICATORS **")
Logger.blank_line(log)

log.info(f"Extraction Directory: {EXTRACTION_DIR}")
log.info(f"Reports Directory: {REPORTS_DIR}")

db = SQLAlchemyClient(CONFIG_ID)

# Load Technical Object Numbers from APM

In [ ]:
# Get : Load Technical Object Numbers from APM for relevant technical objects
# ------------------------------------------------------------------------------ #

# Standard Imports
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# custom imports
from modules.util.database import (
    V_Transform_Indicators,
    ApmTechnicalObjects
)
from modules.apm.explore_technical_objects import ApiTechnicalObjects, APIException
from modules.util.helpers import convert_dataframe

file_tgt = f"{REPORTS_DIR}/APM_Technical_Objects.csv"
file_err = f"{REPORTS_DIR}/APM_Technical_Objects_Errors.csv"

Logger.blank_line(log)
log.info("APM: Get Technical Object Numbers")
Logger.blank_line(log)

log.info(f"Reading data from {V_Transform_Indicators.__tablename__}")
df_objects = pd.DataFrame(db.select(model=V_Transform_Indicators, fields=['externalId'], distinct=True))

api = ApiTechnicalObjects(CONFIG_ID)

def call_api(id):
    return(api.get_technical_object_number(id))

response = []
error = []

with ThreadPoolExecutor(max_workers=20) as executor:
    future_id = {executor.submit(call_api, id): id for id in df_objects['externalId']}

    for future in as_completed(future_id):
        id = future_id[future]
        try:
            res = future.result()
            if res:
                response.append(res)
        except APIException as api_e:
            log.error(f"API Exception: {id} - {api_e.status_code} - {api_e.response} - {api_e.endpoint}")
            err = {
                'id': id,
                'status_code': api_e.status_code,
                'response': api_e.response,
                'endpoint': api_e.endpoint
            }
            error.append(err)
log.info(f"Technical Object Numbers Fetched: {len(response)}")
if error:
    log.error(f"{len(error)} technical objects had errors.")
    df_err = pd.DataFrame(error)
    df_err.to_csv(file_err,index=False)
    log.error(f"{file_err} generated.")

if response:
    df_tech_objects = pd.json_normalize(response)
    df_tech_objects = convert_dataframe(df_tech_objects)
    if '@id' in df_tech_objects.columns:
        df_tech_objects.drop(columns=['@id'], inplace=True)
    
    df_tech_objects.to_csv(file_tgt,index=False)
    log.info(f"{file_tgt} generated")

    if db.drop_reload:
        db.truncate(ApmTechnicalObjects)
    
    technical_objects = SQLAlchemyClient.dataframe_to_object(df_tech_objects, ApmTechnicalObjects)
    if technical_objects:
        db.insert_batches(data=technical_objects)
    else:
        log.warning("Technical object numbers not found in APM")

# Load Indicators to APM

In [ ]:
# Prepare: Load Indicator Data
# ------------------------------------------------------------------------------ #

# standard imports
import pandas as pd

# custom imports
from modules.util.database import (
    V_Transform_Indicators,
    T_UDR_Indicators,
    V_ERPCharacteristics,
    V_APMIndicatorPositions,
    LoadIndicators,
    PreLoadIndicators,
    ApmTechnicalObjects
)

Logger.blank_line(log)
log.info("Load: APM Indicators")
Logger.blank_line(log)

log.info(f"Reading data from {T_UDR_Indicators.__tablename__} / {V_Transform_Indicators.__tablename__} / {V_ERPCharacteristics.__tablename__} / {V_APMIndicatorPositions.__tablename__}")
df_udr = pd.DataFrame(db.select(model=T_UDR_Indicators))
df_indicators = pd.DataFrame(db.select(model=V_Transform_Indicators))
df_chars = pd.DataFrame(db.select(model=V_ERPCharacteristics))
df_positions = pd.DataFrame(db.select(model=V_APMIndicatorPositions))
df_technicalObjects = pd.DataFrame(db.select(model=ApmTechnicalObjects))
log.info(f"Preparing Indicator Data for Migration")
df_final = pd.DataFrame()
df_final = pd.merge(df_udr, df_indicators, on=['tenantid', 'id', 'templateId', 'indicatorGroups_id', 'indicators_id'], suffixes=('', '_udr'), how='left')
df_final = pd.merge(df_final, df_chars, on=['tenantid', 'ERPCharacteristic'], suffixes=('', '_char'), how='left')
df_final = pd.merge(df_final, df_positions, on=['tenantid', 'indicatorGroups_internalId', 'APMIndicatorPosition'], suffixes=('', '_pos'), how='left')

df_final.drop(columns=['idx_pos', 'idx_char', 'idx'], inplace=True)
df_final.drop_duplicates(inplace=True)

df_final['technicalObject_type'] = df_final['objectType'].apply(lambda x: 'EQUI' if x == 'EQU' else 'FLOC' if x == 'FL' else None)
df_final['valid'] = df_final.apply(lambda row: 'X' if row['APMIndicatorCategory'] and row['CharcInternalID'] and row['apm_guid'] and row['ssid'] else None, axis=1)

# consider the technical object number from APM and not externalID for posting data
df_final = pd.merge(df_final, df_technicalObjects[['technicalObject', 'number']], left_on='externalId', right_on='technicalObject', how='left')
df_final.rename(columns={'number': 'APMTechnicalObjectNumber'}, inplace=True)
df_final.drop(columns=['technicalObject'], inplace=True)


if len(df_final) > 0:
    df_final = df_final[['tenantid','internalId', 'name', 'externalId', 'objectType', 'indicatorGroups_internalId', 
                         'indicatorGroups_description_short', 'indicators_internalId', 'indicators_description_short', 'indicators_datatype',
                         'indicators_scale','indicators_precision', 'id', 'templateId', 'indicatorGroups_id', 'indicators_id',
                         'ERPCharacteristic', 'CharcInternalID','APMIndicatorCategory', 'apm_guid','ssid', 'technicalObject_type', 'APMTechnicalObjectNumber','valid']]
    if db.drop_reload:
        log.info(f"Clearing data from {PreLoadIndicators.__tablename__}")
        db.truncate(PreLoadIndicators)
    log.info(f"Updating {PreLoadIndicators.__tablename__}")    
    indicators = SQLAlchemyClient.dataframe_to_object(df_final, PreLoadIndicators)
    db.insert_batches(indicators)
    log.info(f"Updated {PreLoadIndicators.__tablename__} with {db.count(PreLoadIndicators)} records")


df_final = df_final[['tenantid', 'APMTechnicalObjectNumber', 'technicalObject_type', 'APMIndicatorCategory', 'CharcInternalID', 'apm_guid', 'ssid', 'valid']]
df_final.columns = ['tenantid', 'technicalObject_number', 'technicalObject_type', 'category_name', 'characteristics_characteristicsInternalId', 'positionDetails_ID', 'technicalObject_SSID', 'valid']

df_final['category_SSID'] = df_final['technicalObject_SSID']
df_final['characteristics_SSID'] = df_final['technicalObject_SSID']

df_final = df_final.drop_duplicates()

if len(df_final) > 0:
    if db.drop_reload:
        log.info(f"Clearing data from {LoadIndicators.__tablename__}")
        db.truncate(LoadIndicators)
    
    log.info(f"Updating {LoadIndicators.__tablename__}")
    indicators = SQLAlchemyClient.dataframe_to_object(df_final, LoadIndicators)
    db.insert_batches(indicators)
    log.info(f"Updated {LoadIndicators.__tablename__} with {db.count(LoadIndicators)} records")
else:
    log.warning("No indicators found.")

In [ ]:
# Load: Indicator Data to APM
# ------------------------------------------------------------------------------ #

# standard imports
import pandas as pd

# custom imports
from modules.util.database import LoadIndicators, PostLoadIndicators
from modules.apm.manage_indicators import ApiIndicators
from concurrent.futures import ThreadPoolExecutor, as_completed
from modules.util.api import APIException
from modules.util.helpers import convert_dataframe

file_err = rf"{REPORTS_DIR}/3_APM_Indicators_Errors.csv"

df_indicators = pd.DataFrame(db.select(model=LoadIndicators, where=[LoadIndicators.valid == 'X']))
log.info(f"{len(df_indicators)} valid indicators found")

api = ApiIndicators(CONFIG_ID)
response = []
error = []

def call_api(row):
    return(api.create_indicator(row))

with ThreadPoolExecutor(max_workers=20) as executor:
    future_char = {executor.submit(call_api, row): row for _, row in df_indicators.iterrows()}

    for future in as_completed(future_char):
        row = future_char[future]
        try:
            data = future.result()
            response.append(data)
        except APIException as api_e:
            if api_e.status_code == 409:
                log.warning(f"Indicator already exists: ({row['technicalObject_number']},{row['category_name']},{row['positionDetails_ID']},{row['characteristics_characteristicsInternalId']})")
                data = api.search_indicator(
                    technicalObject_number=row['technicalObject_number'],
                    technicalObject_type=row['technicalObject_type'],
                    category_name=row['category_name'],
                    positionDetails_ID=row['positionDetails_ID'],
                    characteristics_characteristicsInternalId=row['characteristics_characteristicsInternalId'],
                    technicalObject_SSID=row['technicalObject_SSID'],
                    category_SSID=row['category_SSID'],
                    characteristics_SSID=row['characteristics_SSID']
                )
                if "value" in data:
                    response.append(data.get("value")[0])
            else:
                log.error(f"API Exception: ({row['technicalObject_number']},{row['category_name']},{row['positionDetails_ID']},{row['characteristics_characteristicsInternalId']}) - {api_e.status_code} - {api_e.response}")
                err = {
                    'technicalObject_number': row['technicalObject_number'],
                    'category_name': row['category_name'],
                    'positionDetails_ID': row['positionDetails_ID'],
                    'characteristics_characteristicsInternalId': row['characteristics_characteristicsInternalId'],
                    'status_code': api_e.status_code,
                    'response': api_e.response,
                    'endpoint': api_e.endpoint
                }
                error.append(err)
if response:
    df_response = pd.json_normalize(response, sep='_')
    df_response = convert_dataframe(df_response)
    df_response = df_response[[col for col in PostLoadIndicators.__table__.columns.keys() if col in df_response.columns]]

    if db.drop_reload:
        log.info(f"Clearing data from {PostLoadIndicators.__tablename__}")
        db.truncate(PostLoadIndicators)
    
    log.info(f"Updating {PostLoadIndicators.__tablename__}")
    indicators = SQLAlchemyClient.dataframe_to_object(df_response, PostLoadIndicators)
    db.insert_batches(indicators)
    log.info(f"Updated {PostLoadIndicators.__tablename__} with {db.count(PostLoadIndicators)} records")

if error:
    df_error = pd.DataFrame(error)
    df_error.to_csv(file_err, index=False)
    log.error(f"{file_err} generated.")